In [ ]:
from collections import OrderedDict
from torch import nn
from skorch import NeuralNet
from skorch.utils import to_numpy
from sklearn.base import TransformerMixin
from braindecode.models import EEGNetv4
from huggingface_hub import hf_hub_download
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer
from moabb.paradigms import MotorImagery
from moabb.datasets import Cho2017
from moabb.evaluations import WithinSessionEvaluation, CrossSessionEvaluation


import numpy as np
import torch
import pandas as pd

In [ ]:
def remove_clf_layers(model: nn.Sequential):
    """
    Remove the classification layers from braindecode models.
    Tested on EEGNetv4, Deep4Net (i.e. DeepConvNet), and EEGResNet.
    """
    new_layers = []
    for name, layer in model.named_children():
        if 'classif' in name:
            continue
        if 'softmax' in name:
            continue
        new_layers.append((name, layer))
    return nn.Sequential(OrderedDict(new_layers))


def freeze_model(model):
    model.eval()
    for param in model.parameters():
        param.requires_grad = False
    return model


In [ ]:
class FrozenNeuralNetTransformer(NeuralNet, TransformerMixin):
    def __init__(
            self,
            *args,
            criterion=nn.MSELoss,  # should be unused
            unique_name=None,  # needed for a unique digest in MOABB
            **kwargs
    ):
        super().__init__(
            *args,
            criterion=criterion,
            **kwargs
        )
        self.initialize()
        self.unique_name = unique_name

    def fit(self, X, y=None, **fit_params):
        return self  # do nothing

    def transform(self, X):
        X = self.infer(X)
        return to_numpy(X)

    def __repr__(self):
        return super().__repr__() + self.unique_name
    
def flatten_batched(X):
    return X.reshape(X.shape[0], -1)

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

class RandomLogisticRegression(LogisticRegression):
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)
        n_features = X.shape[1]
        
        self.coef_ = np.random.randn(
            1 if n_classes == 2 else n_classes,
            n_features
        )
        self.intercept_ = np.random.randn(
            1 if n_classes == 2 else n_classes
        )
        
        return self

In [ ]:
import pickle

# download the model from the hub:
path_kwargs = hf_hub_download(
    repo_id='PierreGtch/EEGNetv4',
    filename='EEGNetv4_Cho2017/kwargs.pkl',
)
path_params = hf_hub_download(
    repo_id='PierreGtch/EEGNetv4',
    filename='EEGNetv4_Cho2017/model-params.pkl',
)
with open(path_kwargs, 'rb') as f:
    kwargs = pickle.load(f)
    module_cls = kwargs['module_cls']
    module_kwargs = kwargs['module_kwargs']

# load the model with pre-trained weights:
torch_module = module_cls(**module_kwargs)
torch_module.load_state_dict(torch.load(path_params, map_location='cpu'))
embedding = freeze_model(remove_clf_layers(torch_module)).double()
# embedding = remove_clf_layers(torch_module).double()

# Integrate the model in a Scikit-learn pipeline:
sklearn_pipeline = Pipeline([
    ('embedding', FrozenNeuralNetTransformer(embedding, unique_name='pretrained_Cho2017')),
    ('flatten', FunctionTransformer(flatten_batched)),
    ('classifier', RandomLogisticRegression()),
])

In [15]:
paradigm = MotorImagery(
    channels=['C3', 'Cz', 'C4'],  # Same as the ones used to pre-train the embedding
    events=['left_hand', 'right_hand'],
    n_classes=2,
    fmin=0.5,
    fmax=40,
    tmin=0,
    tmax=3,
    resample=128
)
datasets = [Cho2017()]
evaluation = WithinSessionEvaluation(
    paradigm=paradigm,
    datasets=datasets,
    overwrite=True,
    suffix='demo',
)

In [ ]:
results = evaluation.process(pipelines=dict(demo_pipeline=sklearn_pipeline))
results

Cho2017-WithinSession:   0%|          | 0/52 [00:00<?, ?it/s]Trials demeaned and stacked with zero buffer to create continuous data -- edge effects present
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 200 events (all good), 0 – 3 s (baseline off), ~7.1 MiB, data loaded,
 'left_hand': 100
 'right_hand': 100>
  warn(f"warnEpochs {epochs}")
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


No hdf5_path provided, models will not be saved.


Cho2017-WithinSession:   2%|▏         | 1/52 [00:03<02:50,  3.35s/it]Trials demeaned and stacked with zero buffer to create continuous data -- edge effects present
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 200 events (all good), 0 – 3 s (baseline off), ~7.1 MiB, data loaded,
 'left_hand': 100
 'right_hand': 100>
  warn(f"warnEpochs {epochs}")


No hdf5_path provided, models will not be saved.


Cho2017-WithinSession:   4%|▍         | 2/52 [00:06<02:44,  3.29s/it]Trials demeaned and stacked with zero buffer to create continuous data -- edge effects present
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 200 events (all good), 0 – 3 s (baseline off), ~7.1 MiB, data loaded,
 'left_hand': 100
 'right_hand': 100>
  warn(f"warnEpochs {epochs}")
Cho2017-WithinSession:   6%|▌         | 3/52 [00:09<02:37,  3.21s/it]

...

Cho2017-WithinSession:  42%|████▏     | 22/52 [01:13<01:38,  3.29s/it]Downloading data from 'https://s3.ap-northeast-1.wasabisys.com/gigadb-datasets/live/pub/10.5524/100001_101000/100295/mat_data/s23.mat' to file '/Users/andresalvarezolmo/mne_data/MNE-gigadb-data/gigadb-datasets/live/pub/10.5524/100001_101000/100295/mat_data/s23.mat'.
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.ap-northeast-1.wasabisys.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
100%|███████████████████████████████████████| 205M/205M [00:00<00:00, 87.9GB/s]
SHA256 hash of downloaded file: c4a01839bc60d44eff43675492c3056c1338d0202f2cb6c72d37ecaaa95c81b0
Use this value as the 'known_hash' argument of 'pooch.retrieve'

No hdf5_path provided, models will not be saved.


/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Cho2017-WithinSession:  44%|████▍     | 23/52 [02:13<09:48, 20.30s/it]Downloading data from 'https://s3.ap-northeast-1.wasabisys.com/gigadb-datasets/live/pub/10.5524/100001_101000/100295/mat_data/s24.mat' to file '/Users/andresalvarezolmo/mne_data/MNE-gigadb-data/gigadb-datasets/live/pub/10.5524/100001_101000/100295/mat_data/s24.mat'.
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.ap-nort

No hdf5_path provided, models will not be saved.


Cho2017-WithinSession:  46%|████▌     | 24/52 [04:01<21:44, 46.60s/it]Downloading data from 'https://s3.ap-northeast-1.wasabisys.com/gigadb-datasets/live/pub/10.5524/100001_101000/100295/mat_data/s25.mat' to file '/Users/andresalvarezolmo/mne_data/MNE-gigadb-data/gigadb-datasets/live/pub/10.5524/100001_101000/100295/mat_data/s25.mat'.
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.ap-northeast-1.wasabisys.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [ ]:
data = {
    "accuracy": [results['score'].mean()],
    "f1": [results["f1"].mean()],
    "recall": [results["recall"].mean()],
    "specificity": [results["specificity"].mean()],
    "precision": [results["precision"].mean()]     
    } 
df = pd.DataFrame(data)
print(df)